In [1]:
import warnings
warnings.filterwarnings('ignore')
import backtesting
from backtesting import Backtest, Strategy
from backtesting.lib import resample_apply
from backtesting.test import GOOG
from backtesting.lib import crossover
backtesting.set_bokeh_output(notebook=False)
import ta
import pandas as pd
import numpy as np

Loading BokehJS ...

In [ ]:
raw_data = pd.read_csv(r'C:\Users\40100171\OneDrive - Anheuser-Busch InBev\Desktop\Stock_market_strategies\data\raw\stock_data_raw_daily.csv.gz', compression='gzip')

In [ ]:
raw_data.drop(columns=['index'], inplace=True)
sample_data = raw_data.loc[raw_data['Stock Name']=='mazdock.NS']
sample_data.index = pd.to_datetime(sample_data['Date'])

In [ ]:
def calculate_ema(data, window):
    return data.ewm(span=window, adjust=False).mean()

def rsi_indicator(data):
    rsi_daily = ta.momentum.rsi(data)
    return rsi_daily.to_numpy()

def ema_indicator(data, time):
    ema = calculate_ema(data, time)
    return ema.to_numpy()

In [ ]:
class rsi_strategy(Strategy):
    
    daily_rsi_cross = 60
    ema_time_period = 20
    
    def init(self):
        self.rsi_daily = self.I(rsi_indicator, self.data.Close.s)
        self.ema_n = self.I(ema_indicator, self.data.Close.s, self.ema_time_period)
#        self.rsi_monthly = resample_apply('M', rsi_indicator, self.data.Close.s)
    
    
    def next(self):

        if(self.position):
            if(self.rsi_daily[-1]<self.daily_rsi_cross or self.data.Close[-1] < self.ema_n[-1]):
                self.position.close()

        else:
            if (self.rsi_daily[-1]> self.daily_rsi_cross and self.rsi_daily[-2]<= self.daily_rsi_cross):
                price = self.data.Close[-1]
                self.buy(sl = price*0.95)

In [ ]:
bt = Backtest(sample_data, rsi_strategy, cash=10000)
stats = bt.run()
bt.plot()
print(stats)

In [ ]:
#Only if we want to oiptimize the strategy edit and use the below
# stats, heatmap = bt.optimize(
#     daily_rsi_cross=range(40, 80, 5),
#     ema_time_period=range(5, 50, 5),
#     maximize='Equity Final [$]',
#     max_tries=200,
#     random_state=0,
#     return_heatmap=True)

In [ ]:
### Run a strategy for all stocks in the universe
a = []
trades = []
for stock in raw_data['Stock Name'].unique().tolist():
    print(stock)
    sample_data = raw_data.loc[raw_data['Stock Name']==stock,]
    sample_data.index = pd.to_datetime(sample_data['Date'])
    bt = Backtest(sample_data, rsi_strategy, cash=10000)
    stats = bt.run()
    
    result = {
        'Ticker': stock,
        'Equity Final [$]': stats['Equity Final [$]'],
        'Return [%]': stats['Return [%]'],
        'Max Drawdown [%]': stats['Max. Drawdown [%]'],
        'Sharpe Ratio': stats['Sharpe Ratio'],
        'Exposure Time [%]': stats['Exposure Time [%]'],
        '# Trades': stats['# Trades'],
        'Win Rate [%]': stats['Win Rate [%]'],
        'Buy & Hold Return [%]': stats['Buy & Hold Return [%]'],
        'Return (Ann.) [%]': stats['Return (Ann.) [%]'],
        
        # Add more metrics as needed
    }
    
    stats["_trades"]['Stock Name'] = stock
    trades.append(stats["_trades"])
    a.append(result)

final_results = pd.DataFrame(a)
final_trades = pd.concat(trades)